In [263]:
import pandas as pd
import numpy as np
from delai.data.local_disk import get_pandas_chunk
from delai.ml_logic.preprocessing import preprocess_X, preprocess_y
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn import tree

In [118]:
df_X = pd.read_csv('../raw_data/initial_X_train.csv')
df_y = pd.read_csv('../raw_data/initial_y_train.csv')

In [119]:
df_X.head()


,Unnamed: 0,FlightDate,Year,Quarter,Month,DayofMonth,DayOfWeek,Airline,Operating_Airline,Marketing_Airline_Network,...,Dest,CRSDepTime,OriginAirportID,OriginCityName,OriginStateName,DestAirportID,DestCityName,DestStateName,CRSArrTime,Distance
0,4324343,2018-07-20,2018,3,7,20,5,Southwest Airlines Co.,WN,WN,...,DAL,1200,12889,"Las Vegas, NV",Nevada,11259,"Dallas, TX",Texas,1640,1067.0
1,28169346,2022-01-02,2022,1,1,2,7,Spirit Air Lines,NK,NK,...,MSY,2110,15304,"Tampa, FL",Florida,13495,"New Orleans, LA",Louisiana,2148,488.0
2,18078508,2020-02-11,2020,1,2,11,2,Air Wisconsin Airlines Corp,ZW,UA,...,ORF,930,13930,"Chicago, IL",Illinois,13931,"Norfolk, VA",Virginia,1245,717.0
3,23669089,2021-02-10,2021,1,2,10,3,United Air Lines Inc.,UA,UA,...,IAH,700,15304,"Tampa, FL",Florida,12266,"Houston, TX",Texas,828,787.0
4,14530840,2020-03-12,2020,1,3,12,4,JetBlue Airways,B6,B6,...,BOS,1324,11278,"Washington, DC",Virginia,10721,"Boston, MA",Massachusetts,1455,399.0


In [120]:
X = preprocess_X(df_X)

✅ preprocess_X() done


In [121]:
y = preprocess_y(df_y)

✅ preprocess_y() done


In [122]:
X = X.drop(columns=[680], axis =1)
X = X.dropna()

In [123]:
X.shape

(10000, 699)

In [124]:
y.shape

(10000,)

In [283]:
clf = DecisionTreeClassifier(max_depth=None, min_samples_leaf=250, random_state=None, splitter='best', min_samples_split=50)
clf.fit(X,y)
score = cross_val_score(clf, X, y, scoring = 'accuracy', cv=5)


In [271]:
print(score.std())
score.mean()

0.0002000000000000224


0.6529

In [272]:
print(clf.predict_proba(X))


[[0.08016878 0.02109705 0.55274262 0.34599156]
 [0.11888112 0.06293706 0.61538462 0.2027972 ]
 [0.06217617 0.0984456  0.67098446 0.16839378]
 ...
 [0.23703704 0.01111111 0.46666667 0.28518519]
 [0.10431655 0.08992806 0.65827338 0.14748201]
 [0.06217617 0.0984456  0.67098446 0.16839378]]


In [273]:
clf.get_depth()

9

In [274]:
clf.get_n_leaves()

31

In [275]:
#tree.get_params(deep=True)

In [276]:
clf.predict(X, check_input=True)

array([2, 2, 2, ..., 2, 2, 2])

In [277]:
clf.predict_proba(X, check_input=True)

array([[0.08016878, 0.02109705, 0.55274262, 0.34599156],
       [0.11888112, 0.06293706, 0.61538462, 0.2027972 ],
       [0.06217617, 0.0984456 , 0.67098446, 0.16839378],
       ...,
       [0.23703704, 0.01111111, 0.46666667, 0.28518519],
       [0.10431655, 0.08992806, 0.65827338, 0.14748201],
       [0.06217617, 0.0984456 , 0.67098446, 0.16839378]])

In [288]:
from sklearn.tree import convert_to_graphviz
import graphviz

graphviz.Source(export_graphviz(tree))

ImportError: cannot import name 'convert_to_graphviz' from 'sklearn.tree' (/home/tamara/.pyenv/versions/3.8.12/envs/delai/lib/python3.8/site-packages/sklearn/tree/__init__.py)

In [279]:
!pip install graphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 3.1 MB/s eta 0:00:00
